In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
SEED_VALUE = 42
LEARNING_RATE = 0.01
num_cores = multiprocessing.cpu_count()

In [3]:
X_train = pd.read_csv("./Dataset/CompressedData.csv")
X_train.head()

,Count,Open,High,Low,Close,Volume,VWAP,new_date,Asset_ID,Target
0,-0.420178,-0.231602,-0.231818,-0.231463,-0.231600,-0.163735,-0.231576,2018-01-01,0,0.000148
1,-0.406696,-0.231558,-0.231774,-0.231420,-0.231556,-0.163475,-0.231532,2018-01-02,0,0.000393
2,-0.412850,-0.231527,-0.231744,-0.231389,-0.231525,-0.163609,-0.231501,2018-01-03,0,0.000549
3,-0.411005,-0.231471,-0.231686,-0.231333,-0.231469,-0.163602,-0.231445,2018-01-04,0,0.000006
4,-0.389464,-0.230984,-0.231194,-0.230852,-0.230981,-0.162833,-0.230958,2018-01-05,0,0.005618


In [4]:
y_train = X_train['Target']
X_train.drop(["Target", 'new_date', 'High', 'Low', 'Close', 'VWAP'], axis = 1, inplace=True)
X_train.head()

,Count,Open,Volume,Asset_ID
0,-0.420178,-0.231602,-0.163735,0
1,-0.406696,-0.231558,-0.163475,0
2,-0.412850,-0.231527,-0.163609,0
3,-0.411005,-0.231471,-0.163602,0
4,-0.389464,-0.230984,-0.162833,0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size = 0.8, random_state = SEED_VALUE)
X_train.shape, X_test.shape

((14481, 4), (3621, 4))

In [6]:
# Fit basic XGBoost, lgbm and KNN and make predictions
model_xgb = XGBRegressor(random_state = 42, n_jobs = -1)
model_xgb.fit(X_train, y_train)
preds_xgb = model_xgb.predict(X_test)

model_lgb = LGBMRegressor()
model_lgb.fit(X_train, y_train)
preds_lgb = model_lgb.predict(X_test)

model_knn = KNeighborsRegressor(n_neighbors = 10)
model_knn.fit(X_train, y_train)
preds_knn = model_knn.predict(X_test)

print("XGB Base Error: ", 10e6*mse(preds_xgb, y_test))
print("LGB Base Error: ", 10e6*mse(preds_lgb, y_test))
print("KNN Base Error: ", 10e6*mse(preds_knn, y_test))

XGB Base Error:  2.443216458345216
LGB Base Error:  2.259477819615519
KNN Base Error:  2.3744811646447754


In [7]:
model_xgb.get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': -1,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 42,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

In [8]:
# Tune the parameters using Grid search
param_grid = {
    "max_depth": list(np.arange(1, 100, step=24)),
    "n_estimators": list(np.arange(1, 500, step=10)),
    "max_leaves": ['0', '1', '2', '3']
}


grid_search = GridSearchCV(model_xgb, param_grid, cv = 5, verbose = 2, scoring='neg_mean_squared_error', n_jobs = num_cores - 1)

In [9]:
%%time
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
[CV] END ..........max_depth=1, max_leaves=0, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=11; total time=   0.1s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=21; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=21; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=31; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=31; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=41; total time=   0.3s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=51; total time=   0.3s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=51; total time=   0.3s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=61; total time=   0.4s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=61; total time=   0.4s
[CV] END .........max_depth=1, max_leaves=0,

[CV] END ..........max_depth=1, max_leaves=0, n_estimators=1; total time=   0.0s
[CV] END ..........max_depth=1, max_leaves=0, n_estimators=1; total time=   0.0s
[CV] END ..........max_depth=1, max_leaves=0, n_estimators=1; total time=   0.0s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=11; total time=   0.1s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=11; total time=   0.1s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=11; total time=   0.1s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=21; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=31; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=31; total time=   0.2s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=41; total time=   0.3s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=41; total time=   0.3s
[CV] END .........max_depth=1, max_leaves=0, n_estimators=51; total time=   0.3s
[CV] END .........max_depth=

[CV] END .........max_depth=1, max_leaves=1, n_estimators=71; total time=   0.6s
[CV] END .........max_depth=1, max_leaves=1, n_estimators=81; total time=   0.6s
[CV] END .........max_depth=1, max_leaves=1, n_estimators=81; total time=   0.6s
[CV] END .........max_depth=1, max_leaves=1, n_estimators=91; total time=   0.9s
[CV] END .........max_depth=1, max_leaves=1, n_estimators=91; total time=   0.7s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=101; total time=   0.6s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=101; total time=   0.8s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=111; total time=   0.9s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=121; total time=   1.0s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=121; total time=   1.0s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=131; total time=   1.0s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=141; total time=   1.0s
[CV] END ........max_depth=1

[CV] END ........max_depth=1, max_leaves=1, n_estimators=111; total time=   0.7s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=121; total time=   0.8s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=121; total time=   0.9s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=131; total time=   1.0s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=131; total time=   1.1s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=141; total time=   1.0s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=151; total time=   1.1s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=151; total time=   1.1s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=161; total time=   1.1s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=171; total time=   1.3s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=171; total time=   1.2s
[CV] END ........max_depth=1, max_leaves=1, n_estimators=181; total time=   1.1s
[CV] END ........max_depth=1

[CV] END ........max_depth=1, max_leaves=2, n_estimators=261; total time=   3.1s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=271; total time=   1.9s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=271; total time=   2.0s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=281; total time=   2.0s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=291; total time=   2.0s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=291; total time=   2.2s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=301; total time=   2.2s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=301; total time=   2.1s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=311; total time=   2.2s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=321; total time=   2.3s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=321; total time=   2.1s
[CV] END ........max_depth=1, max_leaves=2, n_estimators=331; total time=   2.2s
[CV] END ........max_depth=1

...

[CV] END ........max_depth=97, max_leaves=0, n_estimators=61; total time=   0.7s
[CV] END ........max_depth=97, max_leaves=0, n_estimators=71; total time=   0.6s
[CV] END ........max_depth=97, max_leaves=0, n_estimators=71; total time=   0.7s
[CV] END ........max_depth=97, max_leaves=0, n_estimators=81; total time=   0.8s
[CV] END ........max_depth=97, max_leaves=0, n_estimators=91; total time=   0.8s
[CV] END ........max_depth=97, max_leaves=0, n_estimators=91; total time=   0.7s
[CV] END .......max_depth=97, max_leaves=0, n_estimators=101; total time=   0.9s
[CV] END .......max_depth=97, max_leaves=0, n_estimators=101; total time=   0.9s
[CV] END .......max_depth=97, max_leaves=0, n_estimators=111; total time=   1.1s
[CV] END .......max_depth=97, max_leaves=0, n_estimators=121; total time=   0.8s
[CV] END .......max_depth=97, max_leaves=0, n_estimators=121; total time=   0.7s
[CV] END .......max_depth=97, max_leaves=0, n_estimators=131; total time=   0.9s
[CV] END .......max_depth=97

[CV] END .......max_depth=97, max_leaves=0, n_estimators=491; total time=   3.3s
[CV] END .........max_depth=97, max_leaves=1, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=97, max_leaves=1, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=97, max_leaves=1, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=97, max_leaves=1, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=97, max_leaves=1, n_estimators=1; total time=   0.1s
[CV] END ........max_depth=97, max_leaves=1, n_estimators=11; total time=   0.1s
[CV] END ........max_depth=97, max_leaves=1, n_estimators=11; total time=   0.1s
[CV] END ........max_depth=97, max_leaves=1, n_estimators=11; total time=   0.1s
[CV] END ........max_depth=97, max_leaves=1, n_estimators=11; total time=   0.1s
[CV] END ........max_depth=97, max_leaves=1, n_estimators=11; total time=   0.1s
[CV] END ........max_depth=97, max_leaves=1, n_estimators=21; total time=   0.3s
[CV] END ........max_depth=9

[CV] END .......max_depth=97, max_leaves=1, n_estimators=441; total time=   2.9s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=451; total time=   2.7s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=451; total time=   3.1s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=461; total time=   2.6s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=461; total time=   3.7s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=471; total time=   3.2s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=471; total time=   3.5s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=481; total time=   3.1s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=481; total time=   3.7s
[CV] END .......max_depth=97, max_leaves=1, n_estimators=491; total time=   3.9s
[CV] END .........max_depth=97, max_leaves=2, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=97, max_leaves=2, n_estimators=1; total time=   0.1s
[CV] END .........max_depth=

[CV] END .......max_depth=97, max_leaves=2, n_estimators=331; total time=   2.2s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=331; total time=   2.3s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=341; total time=   2.7s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=351; total time=   2.6s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=351; total time=   2.7s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=361; total time=   2.7s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=371; total time=   2.1s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=371; total time=   2.7s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=381; total time=   2.4s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=381; total time=   2.2s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=391; total time=   2.2s
[CV] END .......max_depth=97, max_leaves=2, n_estimators=391; total time=   2.9s
[CV] END .......max_depth=97

{'max_depth': 1, 'max_leaves': '0', 'n_estimators': 31}

In [10]:
best_xgb_preds = grid_search.predict(X_test)
print("XGB Tuned: ", 10e6*mse(best_xgb_preds, y_test))

XGB Tuned:  2.2624253456167858


In [11]:
model_lgb = LGBMRegressor()
model_lgb.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [12]:
param_grid = {
    "max_depth": list(np.arange(1, 100, step=24)),
    "n_estimators": list(np.arange(1, 500, step=10)),
    "max_leaves": ['0', '1', '2', '3']
}


grid_search = GridSearchCV(model_lgb, param_grid, cv = 5, verbose = 2, scoring='neg_mean_squared_error', n_jobs = num_cores - 1)

In [14]:
best_lgb_preds = grid_search.predict(X_test)
print("lgb Tuned: ", 10e6*mse(best_lgb_preds, y_test))

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.